In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import tensorflow as tf

In [ ]:
raw_data=pd.read_csv('../input/used-bikes-prices-in-india/Used_Bikes.csv')
data=raw_data.copy()
data=data.drop('bike_name',axis=1)
data=data.drop('city',axis=1)
data=data.drop('owner',axis=1)
data=data.drop('age',axis=1)
 

In [ ]:
data_wdummies= pd.get_dummies(data,drop_first=True)
tagrets = data['price']
inputs = data_wdummies.drop('price',axis=1)
scaledInputs=preprocessing.scale(inputs)
scaledTagrets=preprocessing.scale(tagrets)
 

In [ ]:
shuffled_indices =np.arange(scaledInputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs=scaledInputs[shuffled_indices]
shuffled_targets=scaledTagrets[shuffled_indices]

In [ ]:
samples_count=shuffled_inputs.shape[0]
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
 
test_samples_count = samples_count - train_samples_count - validation_samples_count

 
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]
 
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]
 
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]
 

In [ ]:
np.savez('bikes_train', inputs=train_inputs, targets=train_targets)
np.savez('bikes_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('bikes_test', inputs=test_inputs, targets=test_targets)

In [ ]:
npz=np.load('bikes_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('bikes_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('bikes_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

In [ ]:
train_inputs.shape[1]

In [ ]:
input_size = 24
output_size = 1
hidden_layer_size = 100


model = tf.keras.Sequential([
tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
tf.keras.layers.Dense(hidden_layer_size,activation='relu'),

tf.keras.layers.Dense(hidden_layer_size,activation='relu'),

tf.keras.layers.Dense(output_size, activation='relu') # output layer
])


model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


batch_size = 100

max_epochs = 100


early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_inputs, 
train_targets,  
batch_size=batch_size,  
epochs=max_epochs, 
callbacks=[early_stopping],  

validation_data=(validation_inputs, validation_targets),  
verbose = 2  
)  

In [ ]:
testLoss,testAccuracy = model.evaluate(test_inputs,test_targets)